In [ ]:
import cx_Oracle
import requests as req
from bs4 import BeautifulSoup as bs

# ✅ DB 연결 정보 설정
sid = 'XE'
host = '210.119.14.76'
port = 1521
username = 'kiosk'
password = '12345'

# 🔹 오라클 DB 연결
conn = cx_Oracle.connect(f'{username}/{password}@{host}:{port}/{sid}')
cursor = conn.cursor()

# 🔹 크롤링한 데이터 리스트
db_name_list = []
db_exp_list = []
db_img_list = []

url = "https://ediya.com/inc/ajax_brand.php"
params = {"gubun": "menu_more", "product_cate": "7", "chked_val": "", "skeyword": "", "page": 1}
page = 1

while True:
    print(f'{page}페이지 실행', '\n')
    params["page"] = page
    response = req.get(url, params=params)

    if response.status_code != 200 or "데이터 끝" in response.text or page == 23:
        break

    soup = bs(response.text, 'html.parser')

    # 🔹 메뉴 이름
    names = [name.text.strip().split('<span')[0] for name in soup.find_all("h2")]
    db_name_list.extend(names)

    # 🔹 메뉴 설명
    details = [desc.text.replace('\\u200b', '').replace('\\xa0', '') for desc in soup.find_all("div", class_="detail_txt")]
    db_exp_list.extend(details)

    # 🔹 이미지 링크
    images = ['https://ediya.com' + img['src'] for img in soup.find_all("img", alt="")]
    db_img_list.extend(images)

    page += 1

print("✅ 크롤링 완료!")

# ✅ DB 저장을 위한 데이터 준비
menu_data = []
cursor.execute("SELECT NVL(MAX(menu_id), 0) + 1 FROM MENU")  # 현재 최대 ID 가져오기
menu_id_start = cursor.fetchone()[0]

for i in range(len(db_name_list)):
    menu_data.append((menu_id_start + i, db_name_list[i], db_exp_list[i], None, "커피", db_img_list[i]))  # 가격은 NULL 처리

# 🔹 INSERT SQL 실행
insert_query = """
    INSERT INTO MENU (menu_id, menu_name, exp, menu_price, category, image)
    VALUES (:1, :2, :3, :4, :5, :6)
"""
cursor.executemany(insert_query, menu_data)

# 🔹 커밋 후 종료
conn.commit()
cursor.close()
conn.close()

print("✅ 데이터 저장 완료!")


1페이지 실행 

2페이지 실행 

3페이지 실행 

4페이지 실행 

5페이지 실행 

6페이지 실행 

7페이지 실행 

8페이지 실행 

